Task 1

---



---



In [1]:
import nltk
from nltk import *
from nltk.sem import logic
nltk.download('book_grammars')


[nltk_data] Downloading package book_grammars to /root/nltk_data...
[nltk_data]   Package book_grammars is already up-to-date!


True

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
from nltk import data, ChartParser

grammar_string = str("""% start S
# Grammar Rules
S[SEM = <app(?subj,?vp)>] -> NP[NUM=?n, SEM=?subj] VP[NUM=?n, SEM=?vp]
NP[NUM=?n, SEM=<app(?det, ?nom)>] -> Det[NUM=?n, SEM=?det] Nom[NUM=?n, SEM=?nom]
NP[LOC=?l, NUM=?n, SEM=?np] -> PropN[LOC=?l, NUM=?n, SEM=?np]
NP[-LOC, NUM=sg, SEM=<\Q. (- exists x. (person(x) & Q(x)))>] -> 'nobody' | 'Nobody'
NP[-LOC, NUM=sg, SEM=<\Q. exists x. (person(x) & Q(x))>] -> 'somebody' | 'Somebody'
Nom[NUM=?n, SEM=<app(?adj,?nom)>] -> Adj[NUM=?n, SEM=?adj] Nom[NUM=?n, SEM=?nom]
Nom[NUM=?n, SEM=?nom] -> N[NUM=?sg, SEM=?nom]
VP[NUM=?n, SEM=<app(?v, ?obj)>] -> TV[NUM=?n, SEM=?v] NP[SEM=?obj]
VP[NUM=?n, SEM=?v] -> IV[NUM=?n, SEM=?v]
VP[NUM=?n, SEM=<app(?pp, ?vp)>] -> VP[NUM=?n,SEM=?vp] PP[-PRED, SEM=?pp]
VP[SEM=<app(?v, ?obj)>] -> TV[SEM=?v] NP[SEM=?obj]
VP[SEM=?v] -> IV[SEM=?v]
VP[SEM=<app(?v, ?obj)>] -> TV[SEM=?v] NP[SEM=?obj]
VP[SEM=?v] -> IV[SEM=?v]
# Lexical Rules
N[NUM=sg,SEM=<\\x.it(x)>] -> 'it'
N[NUM=sg,SEM=<\\x.kitten(x)>] -> 'kitten'
N[NUM=pl,SEM=<\\x.kitten(x)>] -> 'kittens'
N[NUM=sg,SEM=<\\x.mouse(x)>] -> 'mouse'
N[NUM=pl,SEM=<\\x.mouse(x)>] -> 'mouses'
N[NUM=sg,SEM=<\\x.jar(x)>] -> 'jar'
Det[NUM=sg,SEM=<\P Q.exists x.(P(x) & Q(x))>] -> 'a' | 'A'
Det[NUM=sg,SEM=<\P Q.(- exists x.(P(x) & Q(x)))>] -> 'no' | 'No'
Det[NUM=sg,SEM=<\P Q.exists x.((P(x) & Q(x)) & all y.(P(y) -> (x = y)))>] -> 'the' | 'The'
Adj[SEM=<\\x.black(x)>] -> 'black'
Adj[SEM=<\\x.white(x)>] -> 'white'
TV[num=pl,SEM=<\X y.X(\\x.follow(y,x))>,tns=past] -> 'followed'
TV[NUM=pl,SEM=<\X y.X(\\x.saw(y,x))>,tns=past] -> 'saw'
TV[NUM=pl,SEM=<\X y.X(\\x.follow(y,x))>,tns=pres] -> 'follow'
TV[SEM=<\X y.X(\\x.got_out_of(y,x))>,tns=past] -> 'got out of'
TV[SEM=<\X y.X(\\x.rushed_after(y,x))>,tns=past] -> 'rushed after'
TV[SEM=<\X y.X(\\x.jumped_into(y,x))>,tns=past] -> 'jumped into'
IV[NUM=pl,SEM=<\\x.rush(x)>,tns=pres] -> 'rush'
IV[NUM=pl,SEM=<\\x.jump(x)>,tns=pres] -> 'jump'
IV[SEM=<\\x.run(x)>,tns=past] -> 'ran away'
IV[NUM=pl,SEM=<\\x.run(x)>,tns=pres] -> 'run'""")

with open('grammar.fcfg', 'w') as f:
    f.write(grammar_string)

from nltk import load_parser
parser = load_parser('file:grammar.fcfg')

dom = {'k', 'm', 'j'}
v = """
    kitten => {k}
    jar => {j}
    mouse => {m}
    black => {k}
    white => {k}
    run => {m}
    saw => {(k, m)}
    follow => {(k, m)}
    rushed_after => {(k, m)}
    jumped_into => {(m, j)}
    got_out_of => {(k, j)}
"""

val = nltk.Valuation.fromstring(v)
m = nltk.Model(dom, val)

g = nltk.Assignment(dom)
texts = [
    ['The', 'kitten', 'rushed after', 'the', 'mouse'],
    ['The', 'mouse', 'jumped into', 'a', 'jar'],
    ['The', 'kitten', 'followed', 'the', 'mouse'],
    ['The', 'mouse', 'ran away'],
]

for text in texts:
    trees = list(parser.parse(text))
    l = trees[0]
    l = str(l.label()['SEM'].simplify())
    print('Simplified Sentence: ')
    print(l)
    l = l.replace("\\", "")
    print(f"sentence is: {m.evaluate(l, g)}")
    print()


Simplified Sentence: 
exists x.(kitten(x) & exists z3.(mouse(z3) & rushed_after(x,z3) & all z2.(mouse(z2) -> (z3 = z2))) & all y.(kitten(y) -> (x = y)))
sentence is: True

Simplified Sentence: 
exists x.(mouse(x) & exists z5.(jar(z5) & jumped_into(x,z5)) & all y.(mouse(y) -> (x = y)))
sentence is: True

Simplified Sentence: 
exists x.(kitten(x) & exists z9.(mouse(z9) & follow(x,z9) & all z8.(mouse(z8) -> (z9 = z8))) & all y.(kitten(y) -> (x = y)))
sentence is: True

Simplified Sentence: 
exists x.(mouse(x) & run(x) & all y.(mouse(y) -> (x = y)))
sentence is: True



In [5]:
trees = list(parser.parse(['A', 'white', 'kitten', 'saw', 'a' ,'mouse']))
l = trees[0]
l = str(l.label()['SEM'].simplify())
l

'exists x.(white(\\x.kitten(x),x) & exists z11.(mouse(z11) & saw(x,z11)))'

In [6]:
g = nltk.Assignment(dom)
m.evaluate('exists x.(white(x) & kitten(x) & exists z1372.(mouse(z1372) & saw(x,z1372)))', g)

True

### Task 2

---



---



In [9]:
drt_grammar = str("""% start S
# Grammar Rules
S[SEM = <app(?subj,?vp)>] -> NP[NUM=?n, SEM=?subj] VP[NUM=?n, SEM=?vp]
NP[NUM=?n, SEM=<app(?det, ?nom)>] -> Det[NUM=?n, SEM=?det] Nom[NUM=?n, SEM=?nom]
NP[LOC=?l, NUM=?n, SEM=?np] -> PropN[LOC=?l, NUM=?n, SEM=?np]
Nom[NUM=?n, SEM=<app(?adj,?nom)>] -> Adj[NUM=?n, SEM=?adj] Nom[NUM=?n, SEM=?nom]
Nom[NUM=?n, SEM=?nom] -> N[NUM=?sg, SEM=?nom]
VP[NUM=?n, SEM=<app(?v, ?obj)>] -> TV[NUM=?n, SEM=?v] NP[SEM=?obj]
VP[NUM=?n, SEM=?v] -> IV[NUM=?n, SEM=?v]
VP[NUM=?n, SEM=<app(?pp, ?vp)>] -> VP[NUM=?n,SEM=?vp] PP[-PRED, SEM=?pp]
VP[SEM=<app(?v, ?obj)>] -> TV[SEM=?v] NP[SEM=?obj]
VP[SEM=?v] -> IV[SEM=?v]
VP[SEM=<app(?v, ?obj)>] -> TV[SEM=?v] NP[SEM=?obj]
# Lexical Rules
PropN[-LOC,NUM=sg,SEM=<\P.(DRS([x],[PRO(x)])+P(x))>] -> 'he'
PropN[-LOC,NUM=sg,SEM=<\P.(DRS([x],[PRO(x)])+P(x))>] -> 'she'
PropN[-LOC,NUM=sg,SEM=<\P.(DRS([x],[PRO(x)])+P(x))>] -> 'it'
N[NUM=sg,SEM=<\\x.DRS([],[kitten(x)])>] -> 'kitten'
N[NUM=pl,SEM=<\\x.DRS([],[kitten(x)])>] -> 'kittens'
N[NUM=sg,SEM=<\\x.DRS([],[mouse(x)])>] -> 'mouse'
N[NUM=pl,SEM=<\\x.DRS([],[mouse(x)])>] -> 'mouses'
N[NUM=sg,SEM=<\\x.DRS([], [jar(x)])>] -> 'jar'
Det[NUM=sg,SEM=<\P Q.((DRS([x],[])+P(x))+Q(x))>] -> 'a' | 'A'
Det[NUM=sg,SEM=<\P Q.(not ((DRS([x],[])+P(x))+Q(x)))>] -> 'no' | 'No'
Det[NUM=sg,SEM=<\P Q.((DRS([x],[PRO(x)])+P(x))+Q(x))>] -> 'the' | 'The'
Adj[SEM=<\\x.DRS([],[black(x)])>] -> 'black'
Adj[SEM=<\\x.DRS([],[white(x)])>] -> 'white'
TV[num=pl,SEM=<\X x.X(\y.DRS([], [follow(x,y)]))>,tns=past] -> 'followed'
TV[NUM=pl,SEM=<\X x.X(\y.DRS([], [saw(x,y)]))>,tns=past] -> 'saw'
TV[NUM=pl,SEM=<\X x.X(\y.DRS([], [follow(x,y)]))>,tns=pres] -> 'follow'
TV[SEM=<\X x.X(\y.DRS([], [got_out_of(x,y)]))>,tns=past] -> 'got out of'
TV[SEM=<\X x.X(\y.DRS([], [rushed_after(x,y)]))>,tns=past] -> 'rushed after'
TV[SEM=<\X x.X(\y.DRS([], [jumped_into(x,y)]))>,tns=past] -> 'jumped into'
IV[NUM=pl,SEM=<\\x.DRS([], [rush(x)])>,tns=pres] -> 'rush'
IV[NUM=pl,SEM=<\\x.DRS([], [jump(x)])>,tns=pres] -> 'jump'
IV[SEM=<\\x.DRS([], [run(x)])>,tns=past] -> 'ran away'
IV[NUM=pl,SEM=<\\x.DRS([], [run(x)])>,tns=pres] -> 'run'
""")

with open('drt.fcfg', 'w') as f:
    f.write(str(drt_grammar))

from nltk.parse import load_parser
from nltk.sem.drt import DrtParser
parser = load_parser('drt.fcfg', logic_parser=DrtParser())

dom = {'k', 'm', 'j'}
v = """
    kitten => {k}
    jar => {j}
    mouse => {m}
    black => {k}
    white => {k}
    run => {m}
    saw => {(k, m)}
    follow => {(k, m)}
    rushed_after => {(k, m)}
    jumped_into => {(m, j)}
    got_out_of => {(k, j)}
    PRO => {m, k, j}
"""

#Here i decided to add PRO in valuation with meaning that pronounces can be used to word kitten, mouse and jar

val = nltk.Valuation.fromstring(v)
m = nltk.Model(dom, val)
read_dexpr = DrtExpression.fromstring

In [10]:
texts = [['The', 'kitten', 'rushed after', 'the', 'mouse'],
         ['The', 'mouse', 'jumped into', 'a', 'jar'],
         ['The', 'kitten', 'followed', 'the', 'mouse'],
         ['The', 'mouse', 'ran away']]

lst = []
sm = None
for text in texts:

    trees = list(parser.parse(text))
    l = trees[0]
    l = str(l.label()['SEM'])
    print(l, end=' ')
    print(m.evaluate(str(read_dexpr(l).simplify().fol()), g))
    read_dexpr(l).pretty_print()
    lst.append(read_dexpr(l).simplify().fol())
    if not sm:
        sm = read_dexpr(l)
    else:
        sm += read_dexpr(l)
    print()

([x,z18],[PRO(x), kitten(x), PRO(z18), mouse(z18), rushed_after(x,z18)]) True
 _____________________ 
| x z18               |
|---------------------|
| PRO(x)              |
| kitten(x)           |
| PRO(z18)            |
| mouse(z18)          |
| rushed_after(x,z18) |
|_____________________|

([x,z22],[PRO(x), mouse(x), jar(z22), jumped_into(x,z22)]) True
 ____________________ 
| x z22              |
|--------------------|
| PRO(x)             |
| mouse(x)           |
| jar(z22)           |
| jumped_into(x,z22) |
|____________________|

([x,z26],[PRO(x), kitten(x), PRO(z26), mouse(z26), follow(x,z26)]) True
 _______________ 
| x z26         |
|---------------|
| PRO(x)        |
| kitten(x)     |
| PRO(z26)      |
| mouse(z26)    |
| follow(x,z26) |
|_______________|

([x],[PRO(x), mouse(x), run(x)]) True
 __________ 
| x        |
|----------|
| PRO(x)   |
| mouse(x) |
| run(x)   |
|__________|



In [11]:
sm.simplify().pretty_print()

 ___________________________ 
| x z18 z22 z26 z28 z29 z30 |
|---------------------------|
| PRO(x)                    |
| kitten(x)                 |
| PRO(z18)                  |
| mouse(z18)                |
| rushed_after(x,z18)       |
| PRO(z28)                  |
| mouse(z28)                |
| jar(z22)                  |
| jumped_into(z28,z22)      |
| PRO(z29)                  |
| kitten(z29)               |
| PRO(z26)                  |
| mouse(z26)                |
| follow(z29,z26)           |
| PRO(z30)                  |
| mouse(z30)                |
| run(z30)                  |
|___________________________|


In [12]:
sm.simplify().eliminate_equality().pretty_print()

 ___________________________ 
| x z18 z22 z26 z31 z32 z33 |
|---------------------------|
| PRO(x)                    |
| kitten(x)                 |
| PRO(z18)                  |
| mouse(z18)                |
| rushed_after(x,z18)       |
| PRO(z31)                  |
| mouse(z31)                |
| jar(z22)                  |
| jumped_into(z31,z22)      |
| PRO(z32)                  |
| kitten(z32)               |
| PRO(z26)                  |
| mouse(z26)                |
| follow(z32,z26)           |
| PRO(z33)                  |
| mouse(z33)                |
| run(z33)                  |
|___________________________|


In [13]:
g = nltk.Assignment(dom)
m.evaluate(str(sm.simplify().eliminate_equality().fol()), g)

True